In [2]:
import sys
sys.path.append('..\\..')
import gdspy
import numpy as np
from importlib import reload
from copy import deepcopy
from QCreator import elements
from QCreator import general_sample_creator as creator
from QCreator import meshing
from matplotlib import pyplot as plt
reload(gdspy)

### to have 50 Oms impedance with eps=11.75
tl_core = 20.
tl_gap = 5.
tl_ground = 20.

resonator_core = 20
resonator_gap = 5
resonator_ground = 10

pad_offset = 1100
pad_element_offset = 400
qubit_position_offset = 800

coupler_start = pad_offset + pad_element_offset
coupler_delta = 500
coupler_length = 320
num_couplers = 1

jc = 0.5e-6 # uA/um^2

layers_configuration = {
    'total':0,
    'restricted area':10,
    'for removing':100,
    'JJs':3,
    'air bridges':2,
    'air bridge pads':1,
    'air bridge sm pads':4,
    'vertical gridlines':15,
    'horizontal gridlines':16,
    'inverted':17
}

sample = creator.Sample('SrTiO3-res',layers_configuration,epsilon=15000)


airbridge = elements.AirBridgeGeometry(pad_width = 13,
                                       pad_length = 10,
                                       narrow_width = 5,
                                       narrow_length = 36,
                                       pad_distance = 36,
                                       sm_pad_length = 5,
                                       sm_pad_distance = 36,
                                       layer_configuration = sample.layer_configuration,
                                       bridge_style='misis')



#specify sample vertical and horizontal lengths
sample.chip_geometry.sample_vertical_size=9.7e3
sample.chip_geometry.sample_horizontal_size=9.7e3
central_line_y = sample.chip_geometry.sample_vertical_size/2
chip_edge_ground = elements.ChipEdgeGround(sample.chip_geometry, sample.layer_configuration, sample.pads, 800)
sample.add(chip_edge_ground)

grid_ground = elements.GridGround(sample.chip_geometry, sample.layer_configuration, sample.objects, 
                                  width=4, period=50)
sample.add(grid_ground)

for al_mark_pos in [(0, 0), (3000, 3000), (-3000, 3000), (-3000, -3000), (3000, -3000)]:
    abal1 = elements.AirbridgeAlignmentMarks(position = (central_line_y+al_mark_pos[0], central_line_y+al_mark_pos[1]),
                                             window_size = 300, 
                                             square_size = 80, 
                                             gap = 4,
                                             layer_configuration = sample.layer_configuration)
    sample.add(abal1)

# 1. Create contact pads:
pads_left = []
pads_right = []
for pad_side_id in range(1):
    pad = elements.Pad('pad-left-' + str(pad_side_id),
                       (pad_offset, sample.chip_geometry.sample_vertical_size / 2), np.pi, tl_core,
                       tl_gap, tl_ground,
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry,
                       **elements.default_pad_geometry())
    pads_left.append(pad)
    sample.add(pad)
    pad = elements.Pad('pad-right-' + str(pad_side_id),
                       (sample.chip_geometry.sample_horizontal_size - pad_offset,
                        sample.chip_geometry.sample_vertical_size / 2), 0, tl_core,
                       tl_gap, tl_ground,
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry,
                       **elements.default_pad_geometry())
    pads_right.append(pad)
    sample.add(pad)
    
pads_top = []
pads_bottom = []
for pad_side_id in range(1):
    pad = elements.Pad('pad-bottom-' + str(pad_side_id),
                       (sample.chip_geometry.sample_horizontal_size / 2 * (pad_side_id + 1), pad_offset),
                       -np.pi / 2, tl_core, tl_gap, tl_ground,
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry,
                       **elements.default_pad_geometry())
    pads_bottom.append(pad)
    sample.add(pad)
    pad = elements.Pad('pad-top-' + str(pad_side_id),
                       (sample.chip_geometry.sample_horizontal_size / 2 * (pad_side_id + 1),
                        sample.chip_geometry.sample_vertical_size - pad_offset),
                       np.pi / 2, tl_core, tl_gap, tl_ground,
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry,
                       **elements.default_pad_geometry())
    pads_top.append(pad)
    sample.add(pad)
        
    
pad0 = pads_left[0]
pad1 = pads_right[0]
pad2 = pads_top[0]
pad3 = pads_bottom[0]
    
meander0 = sample.connect_meander(name='meander0', o1=pad0,
                                                 port1='port',
                                                 meander_length=10000,
                                                 length_left=750,
                                                 length_right=750,
                                                 first_step_orientation='left',
                                                 meander_orientation=np.pi/2, meander_type='round',
                                                 min_spacing=100, airbridge=airbridge, r=75)
open0 = sample.open_end(name='open end 0', o=meander0[-1], port='port2')

meander1 = sample.connect_meander(name='meander1', o1=pad1,
                                                 port1='port',
                                                 meander_length=16000,
                                                 length_left=750,
                                                 length_right=750,
                                                 first_step_orientation='right',
                                                 meander_orientation=-np.pi/2, meander_type='round',
                                                 min_spacing=100, airbridge=airbridge, r=75)
open1 = sample.open_end(name='open end 1', o=meander1[-1], port='port2')

meander2 = sample.connect_meander(name='meander2', o1=pad2,
                                                 port1='port',
                                                 meander_length=24000,
                                                 length_left=750,
                                                 length_right=750,
                                                 first_step_orientation='right',
                                                 meander_orientation=0, meander_type='round',
                                                 min_spacing=100, airbridge=airbridge, r=75)
open2 = sample.open_end(name='open end 2', o=meander2[-1], port='port2')

meander3 = sample.connect_meander(name='meander3', o1=pad3,
                                                 port1='port',
                                                 meander_length=34000,
                                                 length_left=750,
                                                 length_right=750,
                                                 first_step_orientation='right',
                                                 meander_orientation=np.pi, meander_type='round',
                                                 min_spacing=100, airbridge=airbridge, r=75)
open3 = sample.open_end(name='open end 3', o=meander3[-1], port='port2')


In [ ]:
sample.draw_design()
sample.watch()

In [13]:
sample.write_to_gds()

Gds file has been writen here:  D:\Documents-cloud\qubits\simulation\QCreator\designs\SrTiO3_resonators\SrTiO3-res.gds


# Frequencies and linewidths of resonances in MHz

In [13]:
sys, connections, elements_ = sample.get_tls(cutoff=2e9)
f, delta, modes = sys.get_modes()
num_modes=18
#f[:num_modes]/(2e6*np.pi), delta[:num_modes]/(2e6*np.pi), f/(2*np.pi*2*delta)

# S11 simulation (phase)

In [14]:
f_ = np.linspace(0.01e9, 8e9, 1601)
# s11 = sample.get_s21('p1', 'p1', f_)
s11 = sample.get_s21('pad-left-0', 'pad-left-0', f_)
s22 = sample.get_s21('pad-left-0', 'pad-left-0', f_)

plt.plot(f_, np.angle(s11))
#plt.plot(f_, s22)
plt.plot(f_, np.angle(s22))

..\..\QCreator\general_sample_creator.py:383: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  s.append(np.linalg.lstsq(m, boundary)[0][a2[0]])


KeyboardInterrupt: 